In [2]:
import gensound
from gensound.signals import Sine, Sawtooth, Square, Triangle, WhiteNoise, Silence, PinkNoise, Mix, WAV
from gensound.effects import OneImpulseReverb, Vibrato
from gensound.filters import SimpleBandPass, SimpleHPF, SimpleLPF, SimpleHighShelf, SimpleLowShelf
from gensound.transforms import ADSR, Fade, Amplitude, CrossFade
from gensound.curve import SineCurve

In [5]:
WAV("../resources/samples/idea_1.mp3").play()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [33]:
import json
class Synth():
    def __init__(self, preset='default'):
        self.preset = preset
        if self.preset == 'default':
            # load json file containing default settings
            data = json.load(open('../presets/default.json'))
        self.__dict__ = data
    def _setparam(self, param, value):
        setattr(self, param, value)

    def shift_freq_by_cents(self, freq, cents):
        return freq*(10**(cents/(1200*3.322038403)))

    def semitones_to_cents(self, semitones):
        return semitones*100

    def cents_to_semitones(self, cents):
        return cents/100

    def detuned_voices(self, pitch, duration, detune_range, n_voices, wave_type='square'):
        # n_voices = how many oscillators in the array
        # detune_range = the difference in cents between the highest and lowest oscillators in the array
        all_cents = [i*detune_range/n_voices - detune_range/2 for i in range(n_voices)] # how much to detune each signal in the array
        # print(isinstance(pitch, str))
        match wave_type: # REQUIRES PYTHON 3.10
            case 'sine':
                f = Sine
            case 'square':
                f = Square
            case 'triangle':
                f = Triangle
            case 'sawtooth':
                f = Sawtooth
        if isinstance(pitch, str):
            # print([f"{pitch}{round(cents):+}" for cents in all_cents])
            return gensound.mix([f(f"{pitch}{round(cents):+}", duration) for cents in all_cents])
        elif (isinstance(pitch, int)) or isinstance(pitch, int):
            return gensound.mix([f(self.shift_freq_by_cents(pitch,cents),duration) for cents in all_cents])

    def generate_audio(self, pitch,duration):
        # self._update()
        self.pitch = pitch
        # 2 LFOS for amplitude modulation
        self.lfo1 = SineCurve(frequency=self.lfo1_freq, depth=self.lfo1_depth, baseline=0.5, duration=duration)
        self.lfo2 = SineCurve(frequency=self.lfo2_freq, depth=self.lfo2_depth, baseline=0.5, duration=duration)

        # 2 oscillators for the main sound
        self.osc1 = self.detuned_voices(self.pitch, duration=duration,detune_range = self.osc1_detune, n_voices = self.n_voices, wave_type=self.osc1_wave)
        self.osc2 = self.detuned_voices(self.pitch, duration=duration,detune_range = self.osc2_detune, n_voices = self.n_voices, wave_type=self.osc2_wave)
        # 1 oscillator for the sub bass
        self.sub_bass = self.detuned_voices(self.sub_freq, duration=duration,detune_range = self.sub_detune, n_voices = self.n_voices, wave_type='sine')
        # 1 oscillator for the high frequency noise
        self.high_freq_noise = WhiteNoise(duration=duration)
        # 1 oscillator for the low frequency noise
        self.low_freq_noise = PinkNoise(duration=duration)
        # Filters

        # EFFECTS
        #   - Reverb
        #   - Vibrato

        # AMPLITUDE ENVELOPES
        #   - 2 ADSR envelopes
        #   - Fade in and out

        # apply lfo1 and lfo2 to osc1 and osc2 amplitude
        self.lfo1 = self.osc1 * self.lfo1
        self.lfo1 = self.lfo1 * ADSR(attack=self.attack, decay=self.decay, sustain=self.sustain, release=self.release)
        self.lfo2 = self.osc2 * self.lfo2
        self.lfo2 = self.lfo2 * ADSR(attack=self.attack, decay=self.decay, sustain=self.sustain, release=self.release)

        self.output = self.osc1 + self.osc2 + self.sub_bass + self.high_freq_noise + self.low_freq_noise

        self.output *= Fade(is_in=True, duration=2000)
        self.output *= Fade(is_in=False, duration=2000)
        self.output = self.output * ADSR(attack=self.attack, decay=self.decay, sustain=self.sustain, release=self.release)
        
        return self.output

In [34]:
synth = Synth()
s = synth.generate_audio('A3',5e3)
s.play()

In [39]:
print(synth.automation)
print(synth.automation['filter1_freq_type'])

{'filter1_freq_type': 'constant', 'filter1_freq_rate': 0, 'filter2_freq_type': 'constant', 'filter2_freq_rate': 0, 'filter3_freq_type': 'constant', 'filter3_freq_rate': 0}
constant


In [40]:
notes = ['C3', 'D3', 'E3', 'G3', 'A3']
s = synth.generate_audio(notes[0],5e3)
for note in notes[1:]:
    s = s | CrossFade(duration=0.5e3) | synth.generate_audio(note,5e3)

In [41]:
s.play()

In [42]:
l=['A3 B3 C3']

In [46]:
l[0].split(' ')[0]

'A3'

In [53]:
chords[1:][0].split(' ')

['A3', 'C3', 'E3']

In [74]:
chords = ["C3 E3 G3", "A3 C3 E3"]
duration_secs=5
crossfade_duration = 0.25

layer1 = synth.generate_audio(chords[0].split(' ')[0],1e3*duration_secs)
layer2 = synth.generate_audio(chords[0].split(' ')[1],1e3*duration_secs)
layer3 = synth.generate_audio(chords[0].split(' ')[2],1e3*duration_secs)
for chord in chords[1:]:
    layer1 = layer1 | CrossFade(duration=1e3*crossfade_duration) | synth.generate_audio(chord.split(' ')[0],1e3*duration_secs)
    layer2 = layer2 | CrossFade(duration=1e3*crossfade_duration) | synth.generate_audio(chord.split(' ')[1],1e3*duration_secs)
    layer3 = layer3 | CrossFade(duration=1e3*crossfade_duration) | synth.generate_audio(chord.split(' ')[2],1e3*duration_secs)

s = 0.5*layer1 + 0.25*layer2 + 0.25*layer3
s *= ADSR(attack=1e3, decay=0.5e3, sustain=5e3, release=1.5e3)
s.play()

In [32]:
from gensound import Sine, Triangle, Square, Pan

sig = Triangle # Sine? Square?

beat = 0.5e3 # 120 bpm
fermata = 0.1 # make fermatas in the melody slightly longer
pause = 0.6 # and breathe for a moment before starting the next phrase

S = sig(f"r D5 D=2 C#=1 B-13=2 A=1 D E=2 F#-13={2+fermata} r={pause} F#=1 F#=2 F#=1 E=2 F#-13=1 G F#-13=2 E={2+fermata} r={pause} "
        f"D+16=1 E=2 F#-13=1 E=2 D+16=1 B-13 C#=2 D+9={2+fermata} r={pause} A'=1 F#-13=2 D+16=1 E=2 G=1 F#-13 E=2 D=3", beat)
A = sig(f"r A4 B=2 A+16=1 G=2 F#-13=1 F# B-13 A A={2+fermata} r={pause} C#=1 B=2 B=1 B A A A D A A={2+fermata} r={pause} "
        f"B=1 A=2 A=1 B-13 A=0.5 G F#=1 B-13 B A#-13 B={2+fermata} r={pause} A=1 A=2 B=1 A=2 A=1 A B-13 A F#-13=3", beat)
T = sig(f"r F#4-13 F#=2 F#=1 D=2 D=1 D D C#-13 D={2+fermata} r={pause} C#=1 D+16=2 D+16=1 D C#-13 D E A, D C#-13={2+fermata} r={pause} "
        f"F#=1 E=2 D=1 D C#-13 D+16 D G+5 F# F#={2+fermata} r={pause} E=1 F#-13=2 F#=1 E=2 C#-13=1 A B C#-13 D=3", beat)
B = sig(f"r D3 B-16 D F# G B-13 D B-16 G A D,={2+fermata} r={pause} A#'-13=1 B=2 A=1 G#-13 A F#-13 C#-13 D F#-13 A={2+fermata} r={pause} "
        f"B=1 C#-13=2 D=1 G, A B G E F# B,={2+fermata} r={pause} C#'-13=1 D C# B C#-13 B A D G, A D,=3", beat)

chorale = S*Pan(25) + B*Pan(-25) + T*Pan(80) + A*Pan(-80) # position the voices in the stereo field
chorale.play() # can you spot the parallel octaves?